# Pré-Requisitos

## Sweetviz - https://pypi.org/project/sweetviz/

In [ ]:
!pip install sweetviz

## Baixa e Descompacta o Dataset

In [ ]:
url = "http://ec2-100-25-20-177.compute-1.amazonaws.com/coletas/"
# JSON (3 GB):
#filename = "prefeito_rio_2020.json.tar.gz"
# CSV (1 GB):
filename = "prefeito_rio_2020.csv.tar.gz"
!wget {url + filename}
!tar -xvzf $filename

### Total de linhas (registros) no arquivo JSON

In [ ]:
count = 0

for line in open("prefeito_rio_2020.json"):
  count += 1

print(count)

## Copia dataset do Google Drive

In [ ]:
from google.colab import drive
# Caminho do arquivo dentro do Drive
filepath = "_DS/prefeito_rio_2020.csv"
drive.mount('/content/drive')

!cp drive/MyDrive/$filepath .

# Bibliotecas

In [ ]:
import pandas as pd
import sweetviz as sv
import json

# Abre o DataSet

In [ ]:
uri = "prefeito_rio_2020.csv"
ds = pd.read_csv(uri)
ds.head()

# Mostra informações sobre o DataSet

In [ ]:
ds.info()

# Seleciona registros com dados nulos

In [ ]:
ds[ds.isna().any(axis=1)]

# Mostra total de valores nulos por colunas

In [ ]:
ds.isna().sum()

# Mostra as colunas informando sobre valores nulos

In [ ]:
ds.isna().any()

# Lista somente as colunas que possuem valores nulos

In [ ]:
ds.columns[ds.isna().any()].tolist()

# Lista colunas com objetos JSON

In [ ]:
# Função que verifica se determinada string é um objeto json (parâmetro opcional = somente dicionário)
def is_json(myjson, ignore_list=True):
  try:
    json_object = json.loads(myjson)
  except:
    return False
  # Verifica se o objeto json é uma lista
  if (ignore_list and type(json_object) == list):
    return False
  else:
    return True
# Analisa sempre o primeiro índice não nulo da referida coluna, ou 0 quando a coluna é vazia
[column for column in ds.columns if is_json(ds[column][ds[column].first_valid_index() or 0])]

# Análise Exploratória de Dados - Sweetviz

## Dataset inteiro

In [ ]:
report = sv.analyze(ds)
report.show_notebook()

### Exporta relatório para HTML

In [ ]:
report.show_html()

## Colunas com objetos JSON

### Coluna user

Transforma os objetos JSON da coluna em um DataFrame:

In [ ]:
expanded_user = pd.json_normalize(ds.user.apply(json.loads))
expanded_user.head()

Analisa o DataFrame (Sweetviz):

In [ ]:
report_user = sv.analyze(expanded_user)
report_user.show_notebook()

Exporta relatório para HTML:

In [ ]:
report_user.show_html(filepath='SWEETVIZ_REPORT_USER.html')

### Coluna geo

### Coluna coordinates

### Coluna place

### Coluna extended_tweet

Transforma os objetos JSON da coluna em um DataFrame:

In [ ]:
# Transforma dados null em objetos json vazio {} antes de carregar
expanded_ext_tweet = pd.json_normalize(ds.extended_tweet.fillna('{}').apply(json.loads))
expanded_ext_tweet.head()

Analisa o DataFrame (Sweetviz):

In [ ]:
# Converte lista para string json antes de analisar
for column in expanded_ext_tweet.columns:
  expanded_ext_tweet[column] = expanded_ext_tweet[column].apply(lambda x: json.dumps(x) if type(x) == list else x)
report_ext_tweet = sv.analyze(expanded_ext_tweet)
report_ext_tweet.show_notebook()

Exporta relatório para HTML:

In [ ]:
report_ext_tweet.show_html(filepath='SWEETVIZ_REPORT_EXTENDED_TWEET.html')

### Coluna entities

Transforma os objetos JSON da coluna em um DataFrame:

In [ ]:
expanded_entities = pd.json_normalize(ds.entities.apply(json.loads))
expanded_entities.head()

Analisa o DataFrame (Sweetviz):

In [ ]:
report_entities = sv.analyze(expanded_entities)
report_entities.show_notebook(filepath='SWEETVIZ_REPORT_ENTITIES.html')